In [1]:
from copy import deepcopy
import requests

In [2]:
"""
step 1: set all values we do not have to range(1,10)
step 2: pop values already in the sector from lists
step 3: use rows and cols to further eliminate options
step 4: for all lists that have a single element, add them to board
step 5: if in each sector / row / col an element only shows up in one of the notes, add it as an answer
step 6: repeat 2-5 until complete
step 7: rejoice
"""

'\nstep 1: set all values we do not have to range(1,10)\nstep 2: pop values already in the sector from lists\nstep 3: use rows and cols to further eliminate options\nstep 4: for all lists that have a single element, add them to board\nstep 5: if in each sector / row / col an element only shows up in one of the notes, add it as an answer\nstep 6: repeat 2-5 until complete\nstep 7: rejoice\n'

In [66]:
sudoku_string = "034009001602100000080004000940506130003000760000400000000200010090000407025700003"

In [67]:
def display_grid(matrix):
    output = ""
    for i, row in enumerate(matrix):
        if i % 3 == 0 and i != 0:
            output += "-" * 21 + "\n"
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                output += "| " 
            output += (str(val) if isinstance(val, int) else ".") + " "
        output += "\n"
    return output


### Step 1

In [68]:
long_list = [int(digit) for digit in sudoku_string]
sudoku = [long_list[i:i+9] for i in range(0, len(long_list), 9)]
for row in sudoku:
    for i in range(len(row)):
        if row[i] == 0:
            row[i] = list(range(1,10))

### Step 2

In [69]:
def get_boxes(matrix: list[list]) -> list[list[list]]:
    boxes = []
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            box = [matrix[x][j:j+3] for x in range(i, i+3)]
            boxes.append(box)
    return boxes

In [70]:
def undo_boxes(boxes: list[list[list]]) -> list[list]:
    grid = [[0]*9 for _ in range(9)]
    for index, box in enumerate(boxes):
        start_row = (index // 3) * 3 
        start_col = (index % 3) * 3
        for i in range(3):
            for j in range(3):
                grid[start_row + i][start_col + j] = box[i][j]
    return grid

In [71]:
def pop_sector(matrix: list[list]) -> list[list]:
    boxes = get_boxes(matrix)
    for sector in boxes:
        sector_values = [i for sublist in sector for i in sublist if type(i) == int]
        for row in sector:
            for i in range(len(row)):
                if type(row[i]) == list:
                    row[i] = list(set(row[i]).symmetric_difference(sector_values))
    return undo_boxes(boxes)


### Step 3

In [72]:
def transpose(matrix: list[list]) -> list[list]:
    return [list(col) for col in zip(*matrix)]

In [73]:
def pop_rows_cols(matrix: list[list]) -> list[list]:
    matrix_copy = deepcopy(matrix)
    for row_num in range(9):
        for col_num in range(9):
            if type(matrix[row_num][col_num]) == int:
                continue
            row_values = [i for i in matrix_copy[row_num] if type(i) == int]
            col_values = [i for i in transpose(matrix_copy)[col_num] if type(i) == int]
            impossible_nums = list(set(col_values+row_values))
            matrix_copy[row_num][col_num] = [ele for ele in matrix_copy[row_num][col_num] if ele not in impossible_nums]
    return matrix_copy
            

### Step 4

In [74]:
def extract_single_lists(matrix: list[list]) -> list[list]:
    matrix_copy = deepcopy(matrix)
    for row in matrix_copy:
        for i in range(len(row)):
            if type(row[i]) == list and len(row[i]) == 1:
                row[i] = row[i][0]
    return matrix_copy

### Step 5

In [75]:
def fill_unique_sector_notes(matrix: list[list]) -> list[list]:
    def get_values_to_fill(sector):
        result = []
        for sublist in sector:
            for item in sublist:
                if isinstance(item, list):
                    result.extend(item)
        return [i for i in result if result.count(i) == 1]
    boxes = get_boxes(matrix)
    for sector in boxes:
        values_to_fill = get_values_to_fill(sector)
        for value in values_to_fill:
            for row in sector:
                for i in range(len(row)):
                    if type(row[i]) == list and value in row[i]:
                        row[i] = value
    return undo_boxes(boxes)

In [76]:
def fill_unique_line_notes(matrix: list[list], transpose_it: bool) -> list[list]:
    matrix_copy = deepcopy(matrix)
    if transpose_it:
        matrix_copy = transpose(matrix_copy)
    for line in matrix_copy:
        notes = []
        for item in line:
            if isinstance(item, list):
                notes.extend(item)
        values_to_fill = [note for note in notes if notes.count(note) == 1]
        for value in values_to_fill:
            for i in range(len(line)):
                if type(line[i]) == list and value in line[i]:
                    line[i] = value
    if transpose_it:
        return transpose(matrix_copy)
    return matrix_copy

### Step 6

In [77]:
def solve_sudoku(matrix: list[list]):
    counter = 1
    result = deepcopy(matrix)
    while counter < 100:
        result = \
        fill_unique_line_notes(
            fill_unique_line_notes(
                fill_unique_sector_notes(
                    extract_single_lists(
                        pop_rows_cols(
                            pop_sector(
                                result
                            )
                        )
                    )
                )
            , False)
        , True)
        all_solved = True
        for row in result:
            for cell in row:
                if not isinstance(cell, int):
                    all_solved = False
                    break
            if not all_solved:
                break
        if all_solved:
            print(f"solved in {counter} iterations \n\n")
            print(display_grid(result))
            return result
        counter += 1  
    return result


In [80]:
print(display_grid(sudoku))

. 3 4 | . . 9 | . . 1 
6 . 2 | 1 . . | . . . 
. 8 . | . . 4 | . . . 
---------------------
9 4 . | 5 . 6 | 1 3 . 
. . 3 | . . . | 7 6 . 
. . . | 4 . . | . . . 
---------------------
. . . | 2 . . | . 1 . 
. 9 . | . . . | 4 . 7 
. 2 5 | 7 . . | . . 3 



In [81]:
result = solve_sudoku(sudoku)

solved in 3 iterations 


2 3 4 | 6 7 9 | 6 5 1 
6 5 2 | 1 9 3 | 9 4 4 
1 8 9 | 6 2 4 | 3 5 2 
---------------------
9 4 7 | 5 7 6 | 1 3 2 
2 5 3 | 9 1 2 | 7 6 4 
3 5 1 | 4 6 3 | 3 7 9 
---------------------
7 6 6 | 2 4 7 | 3 1 9 
8 9 1 | 6 3 5 | 4 2 7 
4 2 5 | 7 4 1 | 9 9 3 

